In [1]:
import torch
import random
import numpy as np

In [2]:
def fix_seed():
    random.seed(0)
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    torch.backends.cudnn.determenistic=True
    return True

fix_seed()

True